<a href="https://colab.research.google.com/github/PRISM-SING-HEALTH/2025-NCP-Projects/blob/main/NCP_Variant_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [381]:
# Initialise imports
import pandas as pd
import numpy as np

# Base path for excel files
base_path = '/content/drive/MyDrive/Variant_DB/Mock_Shared_Drive/'
local_path = '/content/drive/MyDrive/Variant_DB/Mock_Local_Drive/'

# Define standard columns
standard_columns = [
    'MRN',
    'Patient Name',
    'Phenotype',
    'Solved Status',
    'Gene',
    'Transcript',
    'Variant',
    'HGVSg',
    'HGVSc',
    'HGVSp'
]

In [382]:
# Importing data into dataframes
print('Importing data...')

try:
  # Lab Cases
  lab_cases_df = pd.read_excel(
      base_path + '/Internal_Drive/Lab_Case_list.xlsx',
      sheet_name = 'Sheet1',
      header = 0,
      usecols = 'F, AF:AI, AL'
  )
  print('Passed.')

  # ATM Summary (fix)
  atm_summary_df = pd.read_excel(
      base_path + '/Internal_Drive/ATM_c.3576G~A_p.Lys1192Lys.xlsx',
      sheet_name='SUMMARY',
      header=None,  # Assuming no headers in the Excel sheet
      usecols='A:B'  # Only read columns A and B
  )
  # Additional filtering due to unconventional formatting
  print('Performing additional filtering...')
  atm_summary_df_filtered = atm_summary_df.iloc[9:16]  # Ensure rows are correctly indexed
  atm_summary_df_filtered.reset_index(drop=True, inplace=True)  # Reset index for clean output

  # Convert to a structured DataFrame
  print('Transforming into structured DataFrame...')
  atm_structured_df = atm_summary_df_filtered.set_index(0).T  # Set column 0 as the header, transpose
  atm_structured_df.reset_index(drop=True, inplace=True)  # Reset index for clean final DataFrame

  # Invitae Summary
  invitae_summary_df = pd.read_excel(
      base_path + '/External_Drive/Invitae_List.xlsx',
      sheet_name = 'Invitae list header',
      header = 0,
      usecols = 'E:F, M:T'
  )
  print('Passed.')

  # Clinical Summary
  clinical_summary_df = pd.read_excel(
      base_path + '/Internal_Drive/Clinic_List.xlsx',
      sheet_name = '11 Dec',
      header = 0,
      usecols = 'D:E, K')
  print('Passed.')

  # Research Summary
  research_summary_df = pd.read_excel(
      base_path + '/Research_Drive/Research_List.xlsx',
      sheet_name = 'Overall List',
      header = 2,
      usecols = 'B, W:X, AF, AS:AT, AV:AW')
  print('Passed.')

except FileNotFoundError as e:
  print(f"File not found: {e}")
except ValueError as e:
  print(f"Error reading sheet or invalid data format: {e}")

print('Data import complete.')

Importing data...
Passed.
Performing additional filtering...
Transforming into structured DataFrame...
Passed.
Passed.
Passed.
Data import complete.


In [383]:
# Print dataframes
print('Printing dataframes...')

print('\nLab Cases')
print(lab_cases_df)

print('\nATM Summary')
print(atm_structured_df)

print('\nInvitae Summary')
print(invitae_summary_df)

print('\nClinical Summary')
print(clinical_summary_df)

print('\nResearch Summary')
print(research_summary_df)

Printing dataframes...

Lab Cases
                                           Phenotype          Variant_1_gene  \
0  Developmental / Behavioural - Hyperactivity; N...  chr6q14.2q15 (3.72 Mb)   
1  Developmental / Behavioural - Global developme...                   PRRT2   
2  Developmental / Behavioural - Global developme...                  STING1   
3  Developmental / Behavioural - Intellectual dis...                     NaN   
4  Developmental / Behavioural - Delayed fine mot...                   NTRK1   
5  Developmental / Behavioural - Global developme...                  DYRK1A   
6  Developmental / Behavioural - Delayed fine mot...                     NaN   

                        Variant_1_HGVSg          Variant_1_HGVSc  \
0  NC_000006.12:g. 83707798_87418416del                      NaN   
1                   chr16:g.29813703dup     NM_145239.3:c.649dup   
2           NC_000005.10:g.139480847C>T     NM_198282.4:c.463G>A   
3                                   NaN              

In [384]:
# Standardise columns across dataframes
print('Standardising columns across dataframes...')

# Lab Cases
lab_cases_cols = {
    'Number variants detected'    :'Var Count',
    'Variant_1_gene'              :'Gene',
    'Variant_1_HGVSg'             :'HGVSg',
    'Variant_1_HGVSc'             :'HGVSc',
    'Variant_1_HGVSp'             :'HGVSp',
    'Variant_1_zygosity'          :'Zygosity',
    'Variant_1_inheritance'       :'Inheritance',
    'Variant_1_Validation_Status' :'Solved Status'
}

lab_cases_df.rename(columns = lab_cases_cols, inplace = True)
print('Passed.')

# ATM Summary
atm_structured_df_cols = {
    'HGVS_Genomic_GRCh38/hg38'            :'HSVSg',
    'HGVS_MANE Select_Transcript_RefSeq'  :'Transcript',
    'HGVS_MANE Select_cDNA'               :'HGVSc',
    'HGVS_MANE Select_protein'            :'HGVSp',
    'HUGO gene symbol'                    :'Gene',
}

atm_structured_df.rename(columns = atm_structured_df_cols, inplace = True)
print('Passed.')

# Invitae Summary
invitae_summary_df_cols = {
    'Patient ID (MRN)'  :'MRN',
    'Patient Name'      :'Patient Name',
    'Result'            :'Solved Status',
    'Gene'              :'Gene',
    'Transcript'        :'Transcript',
    'Variant'           :'Variant',
    'HGVSc'             :'HGVSc',
    'Protein Change'    :'HGVSp',
}

invitae_summary_df.rename(columns = invitae_summary_df_cols, inplace = True)
print('Passed.')

# Clinical Summary
clinical_summary_df_cols = {
    'Identification No.'        :'MRN',
    'Medical Prob description'  :'Phenotype',
    'Patient Name'              :'Patient Name'
}

clinical_summary_df.rename(columns = clinical_summary_df_cols, inplace = True)
print('Passed.')

# Research Summary
research_summary_df_cols = {
    'IC No (MRN)'         :'MRN',
    'Name'                :'Patient Name',
    'Candidate gene (1)'  :'Gene',
    'Transcript (1)'      :'Transcript',
    'cDNA (1)'            :'HGVSc',
    'Protein (1)'         :'HGVSp',
    'AUTO STATUS'         :'Solved Status'
}

research_summary_df.rename(columns = research_summary_df_cols, inplace = True)
print('Passed.')



Standardising columns across dataframes...
Passed.
Passed.
Passed.
Passed.
Passed.


In [385]:
# Print dataframes after renaming
print('Printing dataframes...')

print('\nLab Cases')
print(lab_cases_df)

print('\nATM Summary')
print(atm_structured_df)

print('\nInvitae Summary')
print(invitae_summary_df)

print('\nClinical Summary')
print(clinical_summary_df)

print('\nResearch Summary')
print(research_summary_df)

Printing dataframes...

Lab Cases
                                           Phenotype                    Gene  \
0  Developmental / Behavioural - Hyperactivity; N...  chr6q14.2q15 (3.72 Mb)   
1  Developmental / Behavioural - Global developme...                   PRRT2   
2  Developmental / Behavioural - Global developme...                  STING1   
3  Developmental / Behavioural - Intellectual dis...                     NaN   
4  Developmental / Behavioural - Delayed fine mot...                   NTRK1   
5  Developmental / Behavioural - Global developme...                  DYRK1A   
6  Developmental / Behavioural - Delayed fine mot...                     NaN   

                                  HGVSg                    HGVSc  \
0  NC_000006.12:g. 83707798_87418416del                      NaN   
1                   chr16:g.29813703dup     NM_145239.3:c.649dup   
2           NC_000005.10:g.139480847C>T     NM_198282.4:c.463G>A   
3                                   NaN              

In [386]:
# Query item testing

#print('Initialising querying...\n')

#print('Test: Lab Cases\n')
#search_item = 'PRRT2'  # Item to query
#from_category = 'Var1 Gene' # Category to query from
#lab_cases_result = lab_cases_df[lab_cases_df[from_category] == search_item]
#print(lab_cases_result)

#search_item = 'PRRT2'  # Item to query
#from_category = 'Var1 Gene' # Category to query from
#atm_summary_result = lab_cases_df[lab_cases_df[from_category] == search_item]

#print('Test: Invitae Cases\n')
#search_item = 'TSC2'  # Item to query
#from_category = 'Gene' # Category to query from
#invitae_summary_result = invitae_summary_df[invitae_summary_df[from_category] == search_item]
#print(invitae_summary_result)

#print('Test: Clinical Cases')
#search_item = 'VSD'  # Item to query
#from_category = 'Medical Prob Desc.' # Category to query from
#clinical_summary_result = clinical_summary_df[clinical_summary_df[from_category] == search_item]
#print(clinical_summary_result)  # ERROR: Empty dataframe

#print('Test: Research Cases')
#search_item = 'NM_003718.5'  # Item to query
#from_category = 'Transcript (1)' # Category to query from
#research_summary_result = research_summary_df[research_summary_df[from_category] == search_item]  # KeyERROR
#print(research_summary_result)



In [387]:
# Export results to excel
#print('Exporting results...')
#lab_cases_result.to_excel(local_path + 'lab_cases_results.xlsx', index = False)
#clinical_summary_result.to_excel(local_path + 'clinical_summary_results.xlsx', index = False)
#invitae_summary_result.to_excel(local_path + 'invitae_summary_results.xlsx', index = False)
#clinical_summary_result.to_excel(local_path + 'clinical_summary_results.xlsx', index = False)
#research_summary_result.to_excel(local_path + 'research_summary_results.xlsx', index = False)

#print("Results exported successfully!")

In [388]:
# Combining all 5 dataframes into a singluar dataframe.

def standardise_dataframe(df, columns):
  standardised_df = df.reindex(columns = columns, fill_value = np.nan)
  return standardised_df

print('Standardising all dataframes...')

lab_cases_df_standard = standardise_dataframe(lab_cases_df, standard_columns)
atm_structured_df_standard = standardise_dataframe(atm_structured_df, standard_columns)
invitae_summary_df_standard = standardise_dataframe(invitae_summary_df, standard_columns)
clinical_summary_df_standard = standardise_dataframe(clinical_summary_df, standard_columns)
research_summary_df_standard = standardise_dataframe(research_summary_df, standard_columns)

print('Standardisation complete.')

print('Individual dataframes:')
print('\nLab Cases:')
print(lab_cases_df_standard.head())

print('\nATM:')
print(atm_structured_df_standard.head())

print('\nInvitae:')
print(invitae_summary_df_standard.head())

print('\nClinical:')
print(clinical_summary_df_standard.head())

print('\nResearch:')
print(research_summary_df_standard.head())

print('\nCombining dataframes...')
combined_df = pd.concat([
    lab_cases_df_standard,
    atm_structured_df_standard,
    invitae_summary_df_standard,
    clinical_summary_df_standard,
    research_summary_df_standard
], ignore_index = True)

print('All dataframes combined successfully.')

print('Combined dataframe:')
print(combined_df)

Standardising all dataframes...
Standardisation complete.
Individual dataframes:

Lab Cases:
   MRN  Patient Name                                          Phenotype  \
0  NaN           NaN  Developmental / Behavioural - Hyperactivity; N...   
1  NaN           NaN  Developmental / Behavioural - Global developme...   
2  NaN           NaN  Developmental / Behavioural - Global developme...   
3  NaN           NaN  Developmental / Behavioural - Intellectual dis...   
4  NaN           NaN  Developmental / Behavioural - Delayed fine mot...   

                       Solved Status                    Gene  Transcript  \
0  Confirmed present by clinical CMA  chr6q14.2q15 (3.72 Mb)         NaN   
1                  Confirmed present                   PRRT2         NaN   
2                  Confirmed present                  STING1         NaN   
3                                NaN                     NaN         NaN   
4                  Confirmed present                   NTRK1         NaN   
